In [ ]:
import pandas as pd
import numpy as np
import re
from google.colab import files

df_reject = pd.read_csv('/content/rejected_2007_to_2018Q4 2.csv', low_memory=False)
df_states_ref = pd.read_csv('/content/us-state-codes.csv')

#policy code sütununu sildim
last_col_name = df_reject.columns[-1]
print(f"Silinen Son Sütun: {last_col_name}")
df_reject.drop(columns=[last_col_name], inplace=True)

#statelerin açıklamalarını ekledim
state_code_col = df_states_ref.columns[0]
state_name_col = df_states_ref.columns[1]

df_states_ref['clean_code'] = df_states_ref[state_code_col].astype(str).str.replace('US-', '', regex=False)
state_map = dict(zip(df_states_ref['clean_code'], df_states_ref[state_name_col]))

df_reject['State_Description'] = df_reject['State'].map(state_map)
df_reject['State_Description'] = df_reject['State_Description'].fillna(df_reject['State'])

# debt - to - income - ratio % temizliği
if df_reject['Debt-To-Income Ratio'].dtype == 'O':
    df_reject['dti_clean'] = df_reject['Debt-To-Income Ratio'].astype(str).str.replace('%', '', regex=False)
    df_reject['dti_clean'] = df_reject['dti_clean'].str.strip()
    df_reject['dti_clean'] = pd.to_numeric(df_reject['dti_clean'], errors='coerce')
else:
    df_reject['dti_clean'] = df_reject['Debt-To-Income Ratio']

print("\n--- DTI Kontrolü ---")
print("Orijinal:", df_reject['Debt-To-Income Ratio'].iloc[0])
print("Temizlenmiş:", df_reject['dti_clean'].iloc[0])


# Emp Length
def clean_emp_length_guvenli(val):
    if pd.isna(val): return np.nan
    val = str(val).lower().strip()
    if '<' in val: return 0.0
    match = re.search(r'(\d+)', val)
    if match: return float(match.group(1))
    else: return np.nan

df_reject['emp_length_numeric'] = df_reject['Employment Length'].apply(clean_emp_length_guvenli)


df_reject['Application Date'] = pd.to_datetime(df_reject['Application Date'], errors='coerce')
df_reject['Risk_Score'] = pd.to_numeric(df_reject['Risk_Score'], errors='coerce').fillna(0)


print("Son sütun silindi.")
print("Eyalet isimleri (Alaska, California...) eklendi.")
print("DTI yüzdeleri hassas şekilde (ondalık bozulmadan) temizlendi.")

df_reject.to_csv('rejected_loans_cleaned.csv', index=False)
files.download('rejected_loans_cleaned.csv')

Silinen Son Sütun: Policy Code

--- DTI Kontrolü ---
Orijinal: 10%
Temizlenmiş: 10.0
Son sütun silindi.
Eyalet isimleri (Alaska, California...) eklendi.
DTI yüzdeleri hassas şekilde (ondalık bozulmadan) temizlendi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>